In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：为批量预测训练表格二分类模型的AutoML

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看在 GitHub 上
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_batch.ipynb">
      在 Google Cloud Notebooks 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建表格二元分类模型，并使用Google Cloud的AutoML模型进行批量预测。您可以点击[此处](https://cloud.google.com/vertex-ai/docs/start/automl-users)查看更多信息。

数据集

本教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)数据集。该数据集不需要任何特征工程。您将在本教程中使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将从 Python 脚本中创建一个 AutoML 表格二元分类模型，然后使用 Vertex SDK 进行批量预测。您还可以选择使用 `gcloud` 命令行工具或在 Cloud Console 上在线创建和部署模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行即时预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在 Cloud Storage 存储桶中。

费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 运行在带有Python 3的虚拟环境中的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的满足这些要求的说明。以下步骤提供了一套简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新版本的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的谷歌云项目

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。第一次创建账户时，您将获得300美元的免费信用额度，可用于支付计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，则需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并会插入以 `$` 开头的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改 `REGION` 变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的地区。我们建议您选择离您最近的地区。

- 美洲： `us-central1`
- 欧洲： `europe-west4`
- 亚太地区： `asia-east1`

您不能在Vertex AI中使用多地区存储桶进行训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在直播教程会话中，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，则您的环境已经通过验证。跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行操作，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐户访问授予项目**部分，点击角色下拉列表。在筛选框中输入“Vertex”，并选择**Vertex管理员**。在筛选框中输入“Storage Object Admin”，并选择**Storage Object管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您初始化用于Python的 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源的所有数据在会话之间保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下代码来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化Python版Vertex SDK

为您的项目和相应的存储桶初始化Python版Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您可以开始创建自己的AutoML表格二分类模型了。

#### 云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

#### 快速查看数据

本教程使用存储在公共云存储桶中的Bank Marketing数据集的一个版本，使用CSV索引文件。

首先做一个快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行数据。

您还需要知道训练时标签列的标题名称，它保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Bank Marketing" + "_" + TIMESTAMP, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练自动机器学习模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个自动机器学习训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="bank_" + TIMESTAMP,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

运行训练管道

接下来，通过调用方法`run`，以以下参数运行DAG来启动训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列的名称。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则训练可能会在服务认为无法进一步改善模型目标测量之前完成使用整个预算。

完成`run`方法后将返回`Model`资源。

训练管道的执行将最多需要20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 检查模型评价分数
在模型训练完成后，您可以查看其评价分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=bank_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送一个批量预测请求

发送一个批量预测到您部署的模型。

制作测试项目

你将使用合成数据作为测试数据项目。不要担心我们正在使用合成数据——我们只是想展示如何进行预测。

制作批量输入文件

现在制作一个批量输入文件，将其存储在您本地的云存储桶中。与图像、视频和文本不同，表格的批量输入文件仅支持CSV格式。对于CSV文件，您要做的是：

- 第一行是包含特征（字段）标题名称的标题。
- 每个剩余的行是一个单独的预测请求，带有相应的特征值。

例如：

    "feature_1", "feature_2", ...
    value_1, value_2, ...

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = BUCKET_NAME + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法来进行批量预测，需要使用以下参数：

- `job_display_name`：用于批量预测作业的易读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测结果的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `sync`：如果设置为True，则调用将在等待异步批量作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="csv",
    predictions_format="csv",
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud Storage 输出存储桶。您可以调用iter_outputs()方法获取一个包含每个包含结果的 Cloud Storage 文件的列表。每个文件都以CSV格式包含一个或多个预测请求：

- CSV标题 + 预测标签
- CSV行 + 预测结果，每个预测请求一个

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的个别资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME